In [1]:
import pandas as pd
from datetime import timedelta
import requests
import json
import time
from datetime import datetime
from urllib.request import urlopen
import csv
import numpy as np
import plotly.graph_objects as go

ETH = '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1'
WBTC = '0x2f2a2543B76A4166549F7aaB2e75Bef0aefC5B0f'
LINK = '0xf97f4df75117a78c1A5a0DBb814Af92458539FB4'
UNI = '0xFa7F8980b0f1E64A2062791cc3b0871572f1F7f0'

coins = {ETH:'ETH/USD',WBTC:'WBTC/USD'} #maybe include LINK actually for pnds
wbtc_data = []
eth_data = []
today_date = datetime.now().strftime("%m/%d/%Y")

def price_format(x):
    return int(x)/10**30

def included_coins(x):
    return True if x in coins else False

prices_url = "https://api.gmx.io/prices"
response = urlopen(prices_url) # store the response of URL 
prices_json = json.loads(response.read()) # storing the JSON response
# prices_json -> dict : {'addy':'price * 10**30}

def get_prices():
    response = urlopen(prices_url) # store the response of URL 
    prices_json = json.loads(response.read()) # storing the JSON response
    return prices_json

def time_price_data():
    
    p_json = get_prices()
    now = datetime.now()
    current_time = now.strftime("%I:%M:%S")
    print("Time:", current_time)
    
    for addy in p_json:
        if addy in coins:
            # print(coins[addy],price_format(p_json[addy]))

            if coins[addy] == 'WBTC/USD':
                wbtc_data.append([today_date,current_time,'WBTC/USD',price_format(p_json[addy])])
            if coins[addy] == 'ETH/USD':
                eth_data.append([today_date,current_time,'ETH/USD',price_format(p_json[addy])])
    return

In [3]:
go = True
while go:
    time_price_data()
    time.sleep(0.5)

Time: 10:32:46
Time: 10:32:47
Time: 10:32:48
Time: 10:32:49
Time: 10:32:50
Time: 10:32:51
Time: 10:32:53
Time: 10:32:54
Time: 10:32:55
Time: 10:32:56
Time: 10:32:57
Time: 10:32:58
Time: 10:32:59
Time: 10:33:01
Time: 10:33:02
Time: 10:33:03
Time: 10:33:04
Time: 10:33:05
Time: 10:33:06
Time: 10:33:07
Time: 10:33:08
Time: 10:33:09
Time: 10:33:10
Time: 10:33:11
Time: 10:33:12
Time: 10:33:13
Time: 10:33:15
Time: 10:33:15
Time: 10:33:17
Time: 10:33:18
Time: 10:33:19
Time: 10:33:20
Time: 10:33:21
Time: 10:33:22
Time: 10:33:24
Time: 10:33:25
Time: 10:33:26
Time: 10:33:27
Time: 10:33:28
Time: 10:33:29
Time: 10:33:30
Time: 10:33:32
Time: 10:33:32
Time: 10:33:33
Time: 10:33:34
Time: 10:33:36
Time: 10:33:37
Time: 10:33:38
Time: 10:33:39
Time: 10:33:40
Time: 10:33:42
Time: 10:33:43
Time: 10:33:44
Time: 10:33:45
Time: 10:33:46
Time: 10:33:47
Time: 10:33:48
Time: 10:33:49
Time: 10:33:51
Time: 10:33:52
Time: 10:33:53
Time: 10:33:54
Time: 10:33:55
Time: 10:33:56
Time: 10:33:57
Time: 10:33:58
Time: 10:3

KeyboardInterrupt: 

In [4]:
wbtc_data[0:5]

[['10/06/2022', '10:32:46', 'WBTC/USD', 20035.21],
 ['10/06/2022', '10:32:47', 'WBTC/USD', 20035.17],
 ['10/06/2022', '10:32:48', 'WBTC/USD', 20035.05],
 ['10/06/2022', '10:32:49', 'WBTC/USD', 20035.0],
 ['10/06/2022', '10:32:50', 'WBTC/USD', 20035.0]]

In [7]:
col_names = [['Date','Time','Pair','Price']]

#Formatting data and creating an CSV file and dataframe
with open('data.txt','w') as f:
    write = csv.writer(f)
    write.writerows(col_names)
    write.writerows(wbtc_data)

read_file = pd.read_csv('data.txt')
read_file.to_csv('data.csv', index=None)
df = pd.DataFrame(pd.read_csv('data.txt'))

In [9]:
f.close()

In [143]:
def plot_rate(dtt): 
    #will write for btc but ALSO will functionalize
    open = 0
    opens = {}
    closes = {}
    highs = {}
    lows = {}
    average = 0
    range_values = []

    dt = dtt  #time interval(60 for 1m, 300 for 5m)

    # prices = df.loc[:,'Price']
    for i in range(df.shape[0]):
    # for i in range(106):
        t = df.iloc[i,1] #time
        if i+dt <= df.shape[0]: #required ammo for new candle
            dp = df.iloc[i,-1] #new price

            # if i!=0:
            range_values.append(dp)
            #Open and Close values
            if i % dt == 0: #new candle
                open = dp
                opens[t] = open
                if i != 0: #close is last value of interval
                    close = df.iloc[i+dt-1,-1]
                    closes[t] = close
                    lows[t] = np.min(range_values) #---low at end of interval t, NOT WHEN low occurs, important distinction 
                    highs[t] = np.max(range_values)
                    # print('Min/Max:',np.min(range_values),np.max(range_values))
                    # print(i,t,range_values)
                    # print('len:',len(range_values))
                    range_values = [] #reset for next interval
        else:  
            #---Since open is first, there'll be 1 more value in open than close/low/high, so we'll pad close with current open(0 difference)
            gen_time = str((datetime.strptime(list(opens.keys())[-1],'%I:%M:%S') + timedelta(seconds=dt)).time())
            closes[gen_time] = opens[list(opens.keys())[-1]]
            lows[gen_time] = lows[list(lows.keys())[-1]]
            highs[gen_time] = highs[list(highs.keys())[-1]]
            break

    open_data = list(opens.values())
    high_data = list(highs.values())
    low_data = list(lows.values())
    close_data = list(closes.values())
    times = list(opens.keys())


    fig = go.Figure(data=[go.Candlestick(x=times,
                        open=open_data, high=high_data,
                        low=low_data, close=close_data)])

    fig.show()

In [144]:
plot_rate(5)

In [147]:
plot_rate(60)

In [148]:
plot_rate(300)

In [134]:
df.head(60)

,Date,Time,Pair,Price
0,10/06/2022,10:32:46,WBTC/USD,20035.210000
1,10/06/2022,10:32:47,WBTC/USD,20035.170000
2,10/06/2022,10:32:48,WBTC/USD,20035.050000
3,10/06/2022,10:32:49,WBTC/USD,20035.000000
4,10/06/2022,10:32:50,WBTC/USD,20035.000000
5,10/06/2022,10:32:51,WBTC/USD,20035.050000
6,10/06/2022,10:32:53,WBTC/USD,20035.070000
7,10/06/2022,10:32:54,WBTC/USD,20035.790000
8,10/06/2022,10:32:55,WBTC/USD,20035.070000
9,10/06/2022,10:32:56,WBTC/USD,20035.000000
